In [ ]:
import pandas as pd

import shap
import sklearn

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

from raiwidgets import ModelAnalysisDashboard
from responsibleai import ModelAnalysis

### Use boston housing dataset

In [ ]:
boston_dataset = load_boston()
target_feature_name = 'TARGET'
data = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data[target_feature_name] = boston_dataset.target
train_data, test_data = train_test_split(data, test_size=0.2, random_state=7)
treatment_features = ['AGE']
categorical_features = ['CHAS']
continuous_features = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

### Train a model

In [ ]:
X_train = train_data.drop(columns=[target_feature_name])
y_train = train_data[target_feature_name]

model = sklearn.ensemble.RandomForestRegressor()
model.fit(X_train, y_train)

### Compute Responsible AI model and dataset insights

In [ ]:
model_analysis = ModelAnalysis(model, train_data, test_data, target_feature_name, 'regression',
                               categorical_features=categorical_features)

# Queue Responsible AI insights
model_analysis.explainer.add()
model_analysis.counterfactual.add(continuous_features, 10, desired_range=(40, 50))
model_analysis.error_analysis.add()
model_analysis.causal.add(nuisance_model='linear', max_cat_expansion=20, treatment_features=treatment_features)

# Compute insights
print("Computing model analysis...")
model_analysis.compute()
print("Done")

### Visualize insights in the model analysis dashboard

In [ ]:
ModelAnalysisDashboard(model_analysis)